In [1]:
import json
iu_data = "/data/aofei/hallucination/RULE/training/alignment/iuxray.json"
iu_data = json.load(open(iu_data))
iu_data[0]

{'id': 'CXR2079_IM-0711/0.png',
 'image': 'CXR2079_IM-0711/0.png',
 'conversations': [{'from': 'human',
   'value': 'You are provided with a chest X-ray image, a image-related question and 1 reference report(s): Stable cardiomediastinal silhouette. Stable XXXX opacity in the left base, XXXX scarring or atelectasis. Rounded calcified density in the left lung base, XXXX calcified granuloma. No XXXX consolidation. No pleural effusion or pneumothorax. Stable degenerative changes of the spine.\nPlease answer the question based on the image and report and choose from the following two options: [yes, no]. It should be noted that the diagnostic information in the reference reports cannot be directly used as the basis for diagnosis, but should only be used for reference and comparison. Question: Is there any focal airspace opacity present?\n<image>'},
  {'from': 'gpt',
   'value': 'The chest x-ray does not show any focal airspace opacity.'}],
 'rejected_conversations': [{'from': 'human',
   'va

In [2]:
iu_data2 = []
for i in iu_data:
    question_wp = i["conversations"][0]["value"]
    template = i.copy()
    question = question_wp.split("Question:")[-1].strip("\n<image>")
    question = "<image>\n" + question
    template["conversations"][0]["value"] = question
    del template["rejected_conversations"]
    iu_data2.append(template)

iu_data2[0]

{'id': 'CXR2079_IM-0711/0.png',
 'image': 'CXR2079_IM-0711/0.png',
 'conversations': [{'from': 'human',
   'value': '<image>\n Is there any focal airspace opacity present?'},
  {'from': 'gpt',
   'value': 'The chest x-ray does not show any focal airspace opacity.'}]}

In [4]:
len(iu_data2), iu_data2[100]

(1579,
 {'id': 'CXR925_IM-2425/0.png',
  'image': 'CXR925_IM-2425/0.png',
  'conversations': [{'from': 'human',
    'value': '<image>\n Does the chest X-ray image show increased interstitial lung changes?'},
   {'from': 'gpt',
    'value': 'Yes, the chest X-ray image shows increased interstitial lung changes.'}]})

In [9]:
len(iu_data2), iu_data2[48]

(1579,
 {'id': 'CXR499_IM-2116/0.png',
  'image': 'CXR499_IM-2116/0.png',
  'conversations': [{'from': 'human',
    'value': '<image>\n Is the heart size abnormal as seen on the chest X-ray?'},
   {'from': 'gpt',
    'value': 'No, the heart size appears to be normal in the chest X-ray image.'}]})

In [6]:
import json
iu_test_path = "/data/aofei/hallucination/RULE/test/iuxray_test.jsonl"
iu_test = [json.loads(line) for line in open(iu_test_path)]

In [7]:
iu_test[0]

{'question': 'Does the cardiomediastinal silhouette appear normal in the chest X-ray? \n<image>',
 'answer': 'Yes.',
 'image': 'CXR3030_IM-1405/0.png',
 'question_id': 1,
 'text': 'Does the cardiomediastinal silhouette appear normal in the chest X-ray? Please choose from the following two options: [yes, no]\n<image>',
 'report': 'Normal cardiomediastinal silhouette. There is no focal consolidation. There are no XXXX of a large pleural effusion. There is no pneumothorax. There is no acute bony abnormality seen.'}

In [14]:
new_test_data = []
s = 0
for i in iu_test:
    template = dict()
    template['image'] = i['image']
    template['id'] = str(i['question_id']) + "_test" + str(s)
    template["answer_type"] = "CLOSED"
    template['conversations'] = []
    new_qa = {"from": "human", "value": "<image>\n" + i['text'].split("Please choose")[0].strip()}
    new_qa2 = {"from": "gpt", "value": str(i['answer'])}
    template['conversations'] += [new_qa, new_qa2]
    new_test_data.append(template)
    s+=1
    

new_test_data[-1]

{'image': 'CXR49_IM-2110/0.png',
 'id': '2573_test2572',
 'answer_type': 'CLOSED',
 'conversations': [{'from': 'human',
   'value': '<image>\nAre the mediastinal contours within normal limits?'},
  {'from': 'gpt',
   'value': 'Yes, since the report does not mention any abnormalities of the mediastinum.'}]}

In [15]:
iu_train_path = "/data/aofei/hallucination/RULE/training/alignment/IU_Xray/training.json"
iu_test_path = "/data/aofei/hallucination/RULE/training/alignment/IU_Xray/test.json"

with open(iu_train_path, 'w') as train_file:
    json.dump(iu_data2, train_file, indent=2)
with open(iu_test_path, 'w') as test_file:
    json.dump(new_test_data, test_file, indent=2)

In [12]:
s = 0
for i in new_test_data:
    value = i['conversations'][1]['value']
    if value.lower().startswith("yes") or value.lower().startswith("no"):
        continue
    else:
        print(s, value)
        s += 1
s

0 The report does not mention chronic conditions, so the answer based on the report provided would be No or Unable to determine if not strictly considering the information given in the report.
1 The report does not specify, so I cannot provide a yes or no answer based solely on "acute abnormalities."
2 The report does not mention any masses or nodules, so the answer would be no.
3 This information is not provided in the report, so I cannot provide a yes or no answer.
4 
5 The report does not directly state anything about the pulmonary vessels, but the indication that the lungs are clear may suggest that there is no obvious abnormality with the pulmonary vasculature, typically resulting in a "Yes." However, without a specific comment on pulmonary vessels, it is not possible to provide a definitive answer.
6 The report doesn't mention it, so likely no.
7 The report does not mention masses or nodules, suggesting that the answer is likely no, but technically this question cannot be answere

120